# Importing Data

In [1]:
import pandas as pd

df = pd.read_csv('Sample-Superstore - Sample-Superstore.tsv', delimiter = "\t")

In [2]:
for col in df:
    df.rename(columns={col: col.lower().replace(' ', '').replace('-', '')}, inplace=True)

In [3]:
df.head(0)

,rowid,orderid,orderdate,shipdate,shipmode,customerid,customername,segment,country,city,...,postalcode,region,productid,category,subcategory,productname,sales,quantity,discount,profit


In [4]:
# Select relevant columns for each table
orders_cols = ['orderid', 'orderdate', 'shipdate', 'shipmode', 'customerid', 'productid']
customers_cols = ['customerid', 'customername', 'segment']
addresses_cols = ['customerid', 'country', 'city', 'state', 'postalcode', 'region']
products_cols = ['productid', 'category', 'subcategory', 'productname']
sales_cols = ['sales', 'orderid', 'quantity', 'productid', 'discount', 'profit']
prices_cols = ['productid', 'orderdate']

In [5]:
# Create new dataframes for each table
orders = df.loc[:, orders_cols].drop_duplicates(subset='orderid', keep='first').reset_index(drop=True)
customers = df.loc[:, customers_cols].drop_duplicates(subset='customerid', keep='first').reset_index(drop=True)
addresses = df.loc[:, addresses_cols].drop_duplicates(subset='customerid', keep='first').reset_index(drop=True)
products = df.loc[:, products_cols].drop_duplicates(subset='productid', keep='first').reset_index(drop=True)
sales = df.loc[:, sales_cols].reset_index(drop=True)
prices = df.loc[:, prices_cols].drop_duplicates(subset='productid', keep='first').reset_index(drop=True)

In [6]:
def dict_factory(cursor, row):
    fields = [column[0] for column in cursor.description]
    return {key: value for key, value in zip(fields, row)}

In [30]:
import sqlite3
con = sqlite3.connect("Sample-Superstore_01.db")
con.row_factory = dict_factory
cur = con.cursor()

In [31]:
cur.execute("DROP TABLE IF EXISTS Customers")
cur.execute("DROP TABLE IF EXISTS Addresses")
cur.execute("DROP TABLE IF EXISTS Orders")
cur.execute("DROP TABLE IF EXISTS Products")
cur.execute("DROP TABLE IF EXISTS Sales")
cur.execute("DROP TABLE IF EXISTS Prices")

In [32]:
cur.executescript("""
CREATE TABLE Customers (
  customerid TEXT PRIMARY KEY,
  customername TEXT,
  segment TEXT
);
""")

cur.execute("""
CREATE TABLE Orders (
  orderid TEXT PRIMARY KEY,
  orderdate TEXT,
  shipdate DATE,
  shipmode TEXT,
  customerid TEXT,
  productid TEXT,
  FOREIGN KEY (customerid) REFERENCES Customer (customerid)
);
""")

cur.execute("""
CREATE TABLE Products (
  productid TEXT PRIMARY KEY,
  category TEXT,
  subcategory TEXT,
  productname TEXT
);
""")

cur.execute("""
CREATE TABLE Sales (
  orderid TEXT,
  productid TEXT,
  sales FLOAT64,
  quantity INTEGER,
  discount FLOAT64,
  profit FLOAT64,
  FOREIGN KEY (orderid) REFERENCES Orders (orderid)
  FOREIGN KEY (productid) REFERENCES "Products" (productid)
);
""")

cur.execute("""
CREATE TABLE Addresses (
  customerid TEXT PRIMARY KEY,
  country TEXT,
  city TEXT,
  state TEXT,
  postalcode INTEGER,
  region TEXT,
  FOREIGN KEY (customerid) REFERENCES Customer (customerid)
);
""")

cur.execute("""
CREATE TABLE Prices (
  productid TEXT,
  orderdate TEXT,
  FOREIGN KEY (productid) REFERENCES Product (productid),
  FOREIGN KEY (orderdate) REFERENCES Orders (orderdate)
);
""")
con.commit()


In [33]:
orders.info()
customers.info()
addresses.info()
products.info()
sales.info()
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   orderid     5009 non-null   object
 1   orderdate   5009 non-null   object
 2   shipdate    5009 non-null   object
 3   shipmode    5009 non-null   object
 4   customerid  5009 non-null   object
 5   productid   5009 non-null   object
dtypes: object(6)
memory usage: 234.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customerid    793 non-null    object
 1   customername  793 non-null    object
 2   segment       793 non-null    object
dtypes: object(3)
memory usage: 18.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  

In [34]:
sales.head()

,sales,orderid,quantity,productid,discount,profit
0,261.9600,CA-2016-152156,2,FUR-BO-10001798,0.00,41.9136
1,731.9400,CA-2016-152156,3,FUR-CH-10000454,0.00,219.5820
2,14.6200,CA-2016-138688,2,OFF-LA-10000240,0.00,6.8714
3,957.5775,US-2015-108966,5,FUR-TA-10000577,0.45,-383.0310
4,22.3680,US-2015-108966,2,OFF-ST-10000760,0.20,2.5164


In [35]:
orders.to_sql('Orders', con, if_exists='append', index=False)
customers.to_sql('Customers', con, if_exists='append', index=False)
addresses.to_sql('Addresses', con, if_exists='append', index=False)
products.to_sql('Products', con, if_exists='append', index=False)
sales.to_sql('Sales', con, if_exists='append', index=False)
prices.to_sql('Prices', con, if_exists='append', index=False)

1862

In [36]:
def run(cmd):
    r = cur.execute(cmd)
    display(r.fetchall())

In [37]:
# close the connection
# con.close()

# Assignments

What is the category generating the maximum sales revenue?

In [38]:
run("""
SELECT category, SUM(sales)
FROM Sales
LEFT JOIN Products ON Sales.productid = Products.productid
GROUP BY Products.category
ORDER BY SUM(sales) DESC LIMIT 1
;""")

[{'category': 'Technology', 'SUM(sales)': 836154.0329999966}]

What about the profit in this category?


In [39]:
run("""
SELECT Products.category, SUM(profit)
FROM Sales
LEFT JOIN Products ON Sales.productid = Products.productid
GROUP BY Products.category
ORDER BY SUM(sales) DESC LIMIT 1
;""")

[{'category': 'Technology', 'SUM(profit)': 145454.9480999999}]

Are they making a loss in any categroies?

In [40]:
run("""
SELECT Products.category, SUM(profit) as total_profit
FROM Sales
LEFT JOIN Products ON Sales.productid = Products.productid
GROUP BY Products.category
HAVING total_profit < 0
;""")

[]

What are 5 states generating the maximum and minimum sales revenue?

In [41]:
print("Top 5 States:")
run("""
SELECT Addresses.state, SUM(Sales.sales)
FROM Sales
JOIN Orders ON Sales.orderid = Orders.orderid
JOIN Addresses ON Orders.customerid = Addresses.customerid
GROUP BY Addresses.state
ORDER BY SUM(Sales.sales) DESC LIMIT 5
;""")

print("Bottom 5 States:")
run("""
SELECT Addresses.state, SUM(Sales.sales)
FROM Sales
JOIN Orders ON Sales.orderid = Orders.orderid
JOIN Addresses ON Orders.customerid = Addresses.customerid
GROUP BY Addresses.state
ORDER BY SUM(Sales.sales) LIMIT 5
;""")

Top 5 States:


[{'state': 'California', 'SUM(Sales.sales)': 451036.58229999995},
 {'state': 'New York', 'SUM(Sales.sales)': 279549.8235},
 {'state': 'Texas', 'SUM(Sales.sales)': 192758.2048999999},
 {'state': 'Pennsylvania', 'SUM(Sales.sales)': 142838.55100000012},
 {'state': 'Washington', 'SUM(Sales.sales)': 133177.25179999997}]

Bottom 5 States:


[{'state': 'Nevada', 'SUM(Sales.sales)': 1214.986},
 {'state': 'Maryland', 'SUM(Sales.sales)': 1588.81},
 {'state': 'Kansas', 'SUM(Sales.sales)': 1727.652},
 {'state': 'District of Columbia', 'SUM(Sales.sales)': 2198.45},
 {'state': 'South Dakota', 'SUM(Sales.sales)': 2339.598}]

In [42]:
run(""" 
SELECT * FROM (SELECT Addresses.state, SUM(Sales.sales)
FROM Sales
JOIN Orders ON Sales.orderid = Orders.orderid
JOIN Addresses ON Orders.customerid = Addresses.customerid
GROUP BY Addresses.state
ORDER BY SUM(Sales.sales) DESC LIMIT 5)

UNION

SELECT * FROM (SELECT Addresses.state, SUM(Sales.sales)
FROM Sales
JOIN Orders ON Sales.orderid = Orders.orderid
JOIN Addresses ON Orders.customerid = Addresses.customerid
GROUP BY Addresses.state
ORDER BY SUM(Sales.sales) LIMIT 5);
""")

[{'state': 'California', 'SUM(Sales.sales)': 451036.58229999995},
 {'state': 'District of Columbia', 'SUM(Sales.sales)': 2198.45},
 {'state': 'Kansas', 'SUM(Sales.sales)': 1727.652},
 {'state': 'Maryland', 'SUM(Sales.sales)': 1588.81},
 {'state': 'Nevada', 'SUM(Sales.sales)': 1214.986},
 {'state': 'New York', 'SUM(Sales.sales)': 279549.8235},
 {'state': 'Pennsylvania', 'SUM(Sales.sales)': 142838.55100000012},
 {'state': 'South Dakota', 'SUM(Sales.sales)': 2339.598},
 {'state': 'Texas', 'SUM(Sales.sales)': 192758.2048999999},
 {'state': 'Washington', 'SUM(Sales.sales)': 133177.25179999997}]

What are the 3 products in each segment with the highest sales?

In [84]:
run("""
SELECT segment, number, productname 
FROM (SELECT 
ROW_NUMBER() OVER (PARTITION BY segment ORDER BY SUM(sales) DESC) AS number, * 
FROM Orders 
LEFT JOIN Customers ON Orders.customerid = Customers.customerid
LEFT JOIN Sales ON Sales.orderid = Orders.orderid
LEFT JOIN Products ON Orders.productid = Products.productid
GROUP BY segment, Orders.productid)
WHERE number < 4
;""")

[{'segment': 'Consumer',
  'number': 1,
  'productname': 'GBC Ibimaster 500 Manual ProClick Binding System'},
 {'segment': 'Consumer',
  'number': 2,
  'productname': 'Canon imageCLASS 2200 Advanced Copier'},
 {'segment': 'Consumer',
  'number': 3,
  'productname': 'HP Designjet T520 Inkjet Large Format Printer - 24" Color'},
 {'segment': 'Corporate',
  'number': 1,
  'productname': 'Acco Perma 4000 Stacking Storage Drawers'},
 {'segment': 'Corporate',
  'number': 2,
  'productname': '3D Systems Cube Printer, 2nd Generation, Magenta'},
 {'segment': 'Corporate',
  'number': 3,
  'productname': 'Fellowes PB300 Plastic Comb Binding Machine'},
 {'segment': 'Home Office',
  'number': 1,
  'productname': 'Hewlett-Packard Deskjet 6540 Color Inkjet Printer'},
 {'segment': 'Home Office',
  'number': 2,
  'productname': 'Canon imageCLASS 2200 Advanced Copier'},
 {'segment': 'Home Office',
  'number': 3,
  'productname': "Global Deluxe High-Back Manager's Chair"}]

Are they the 3 most profitable products as well?

In [45]:
run("""
SELECT productname, SUM(sales)
FROM Sales
LEFT JOIN Products ON Sales.productid = Products.productid
GROUP BY Products.productid
ORDER BY SUM(sales) DESC LIMIT 3
;""")

[{'productname': 'Canon imageCLASS 2200 Advanced Copier',
  'SUM(sales)': 61599.824},
 {'productname': 'Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind',
  'SUM(sales)': 27453.384},
 {'productname': 'Cisco TelePresence System EX90 Videoconferencing Unit',
  'SUM(sales)': 22638.48}]

What are the 3 best-seller products in each category? (Quantity-wise)

In [95]:
run("""
SELECT category, productname, qnumber
FROM (SELECT 
ROW_NUMBER() OVER (PARTITION BY category ORDER BY SUM(quantity) DESC) AS number, *, SUM(quantity) qnumber  
FROM Orders 
LEFT JOIN Sales ON Sales.orderid = Orders.orderid
LEFT JOIN Products ON Orders.productid = Products.productid
GROUP BY category, Orders.productid)
WHERE number < 4
;""")

[{'category': 'Furniture',
  'productname': 'Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood',
  'qnumber': 105},
 {'category': 'Furniture',
  'productname': 'KI Adjustable-Height Table',
  'qnumber': 85},
 {'category': 'Furniture',
  'productname': 'Global Troy Executive Leather Low-Back Tilter',
  'qnumber': 82},
 {'category': 'Office Supplies',
  'productname': 'Hot File 7-Pocket, Floor Stand',
  'qnumber': 113},
 {'category': 'Office Supplies',
  'productname': 'SAFCO Boltless Steel Shelving',
  'qnumber': 107},
 {'category': 'Office Supplies', 'productname': 'Xerox 191', 'qnumber': 88},
 {'category': 'Technology',
  'productname': 'Sony 64GB Class 10 Micro SDHC R40 Memory Card',
  'qnumber': 88},
 {'category': 'Technology',
  'productname': 'Logitech G19 Programmable Gaming Keyboard',
  'qnumber': 76},
 {'category': 'Technology',
  'productname': "I Need's 3d Hello Kitty Hybrid Silicone Case Cover for HTC One X 4g with 3d Hello Kitty Stylus Pen Green/pink",
  'qnu

What are the top 3 worst-selling products in every category? (Quantity-wise)

In [96]:
run("""
SELECT category, productname, Quantity
FROM (SELECT 
ROW_NUMBER() OVER (PARTITION BY category ORDER BY SUM(quantity)) AS number, *, SUM(quantity) Quantity 
FROM Orders 
LEFT JOIN Sales ON Sales.orderid = Orders.orderid
LEFT JOIN Products ON Orders.productid = Products.productid
GROUP BY category, Orders.productid)
WHERE number < 4
;""")

[{'category': 'Furniture',
  'productname': "O'Sullivan Living Dimensions 5-Shelf Bookcases",
  'quantity': 1},
 {'category': 'Furniture',
  'productname': 'GE General Purpose, Extra Long Life, Showcase & Floodlight Incandescent Bulbs',
  'quantity': 1},
 {'category': 'Furniture',
  'productname': 'Riverside Furniture Stanwyck Manor Table Series',
  'quantity': 1},
 {'category': 'Office Supplies',
  'productname': 'Sanford Colorific Eraseable Coloring Pencils, 12 Count',
  'quantity': 1},
 {'category': 'Office Supplies',
  'productname': 'Avery Binder Labels',
  'quantity': 1},
 {'category': 'Office Supplies',
  'productname': 'Wilson Jones DublLock D-Ring Binders',
  'quantity': 1},
 {'category': 'Technology',
  'productname': 'Rosewill 107 Normal Keys USB Wired Standard Keyboard',
  'quantity': 1},
 {'category': 'Technology',
  'productname': 'Imation\xa0Swivel\xa0Flash Drive\xa0USB\xa0flash drive\xa0- 8 GB',
  'quantity': 1},
 {'category': 'Technology',
  'productname': 'Memorex Min

How many unique customers per month are there for the year 2016. 
(There's a catch here: contrary to other 'heavier' RDBMS, SQLite does not support the functions YEAR() or MONTH() to extract the year or the month in a date. You will have to create two new columns: year and month.)

In [50]:
#Usage by year 
run("""
SELECT DISTINCT count(customerid)
FROM Orders
WHERE shipdate LIKE '%2016'
;""")

[{'count(customerid)': 1305}]

In [51]:
run("""
SELECT substr(shipdate, 1, 2) AS month, COUNT(DISTINCT customerid) AS unique_customers
FROM Orders
WHERE shipdate LIKE '%2016'
GROUP BY month
ORDER BY month;
""")

[{'month': '1/', 'unique_customers': 48},
 {'month': '10', 'unique_customers': 103},
 {'month': '11', 'unique_customers': 164},
 {'month': '12', 'unique_customers': 151},
 {'month': '2/', 'unique_customers': 46},
 {'month': '3/', 'unique_customers': 78},
 {'month': '4/', 'unique_customers': 81},
 {'month': '5/', 'unique_customers': 87},
 {'month': '6/', 'unique_customers': 98},
 {'month': '7/', 'unique_customers': 89},
 {'month': '8/', 'unique_customers': 82},
 {'month': '9/', 'unique_customers': 163}]